* Features engineering
  * Lag (DONE)
  * Lead (DONE)
  * Overall aggregation
  * Rolling aggregation
* Modeling
  * Linear Regression
  * LightGBM -> optimize MAE, MAPE
* Missing imputation
  * Keep
  * Fill with previous + add new column factor

In [ ]:
# Main package
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import StratifiedKFold

# Utility
from itertools import combinations, product
import random
import calendar
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 200)

Let's create a utility functions first

In [ ]:
# Default parameters
TARGET = ['case', 'unit_cost']
SEED = 2021
categorical_features = [
    'kddati2',
    'tkp',
    'id'
]
categorical_features_2 = [
    'a', 'b', 'c', 'cb', 'd', 'ds', 'gd', 'hd', 
    'i1', 'i2', 'i3', 'i4', 
    'kb', 'kc', 'kg', 'ki', 'kj', 'kk', 'kl', 'km', 'ko', 'kp', 'kt', 'ku', 
    's', 'sa', 'sb', 'sc', 'sd'
]
remove_features = ['tglpelayanan','row_id','cat']
numerical_features = ['peserta'] + TARGET + ['case2', 'unit_cost2']

# Utility
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

def mape(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def mae(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred)))

## Data Preparation

In [ ]:
# Read data
train = pd.read_csv('data/case_cost_prediction_train.csv')
test = pd.read_csv('data/case_cost_prediction_val.csv')
df = pd.concat([train.assign(cat = 'Train'), test.assign(cat = 'Test')])
df = df.sort_values(['kddati2','tkp','tglpelayanan'])

# Simple features engineering
df['tglpelayanan'] = pd.to_datetime(df['tglpelayanan'])
df['month'] = df['tglpelayanan'].dt.month
df['year'] = df['tglpelayanan'].dt.year

# Add combined ID
df['id'] = df['kddati2'].astype(str) + '-' + df['tkp'].astype(str)

In [ ]:
# Join with df_temp with different tkp
df_temp = df[['tglpelayanan','kddati2','tkp','case','unit_cost']].copy()
df_temp['tkp'] = np.where(df_temp['tkp'] == 30, 40, 30)
df_temp = df_temp.rename(columns={'case':'case2', 'unit_cost':'unit_cost2'})
df = df.merge(df_temp,how='left')

In [ ]:
# Generate lag/lead features
df = df.assign(**{
        '{}_lag_{}'.format(col, l): df.groupby(['id'])[col].transform(lambda x: x.shift(l))
        for l in [1,-1]
        for col in numerical_features + ['tglpelayanan']
    })
lag_features = [col for col in df.columns if 'lag' in col]

In [ ]:
nlag_features = [col for col in lag_features if 'lag_1' in col and 'tglpelayanan' not in col and 'peserta' not in col]
nlead_features = [col for col in lag_features if 'lag_-1' in col and 'tglpelayanan' not in col and 'peserta' not in col]
print(nlag_features)
print(nlead_features)

In [ ]:
for i, col in enumerate(['case2']):
    df['ds_' + col] = np.where(df[col].isnull(), None, df['tglpelayanan'].astype(str))
    df['ds_' + col] = df.groupby(['id'])['ds_' + col].ffill()
    df['ds_' + col] = np.round(((df['tglpelayanan'] - pd.to_datetime(df['ds_' + col]))/np.timedelta64(1, 'M')))
df[['case2','unit_cost2']] = df.groupby(['id'])['case2','unit_cost2'].ffill()

for i, col in enumerate(['case_lag_1','case2_lag_1']):
    df['ds_' + col] = np.where(df[col].isnull(), None, df['tglpelayanan_lag_1'].astype(str))
    df['ds_' + col] = df.groupby(['id'])['ds_' + col].ffill()
    df['ds_' + col] = np.round(((df['tglpelayanan'] - pd.to_datetime(df['ds_' + col]))/np.timedelta64(1, 'M')))
df[nlag_features] = df.groupby(['id'])[nlag_features].ffill()

for i, col in enumerate(['case_lag_-1','case2_lag_-1']):
    df['ds_' + col] = np.where(df[col].isnull(), None, df['tglpelayanan_lag_-1'].astype(str))
    df['ds_' + col] = df.groupby(['id'])['ds_' + col].bfill()
    df['ds_' + col] = np.round(((df['tglpelayanan'] - pd.to_datetime(df['ds_' + col]))/np.timedelta64(1, 'M')))
df[nlead_features] = df.groupby(['id'])[nlead_features].bfill()


In [ ]:
df = df.drop(columns=['tglpelayanan_lag_1','tglpelayanan_lag_-1'])

In [ ]:
df.head()

## Modeling

In [ ]:
def process_train(
    df,
    target=TARGET[0],
    cv_split=5,
    seed=SEED,
    verbose=500
):
    df = df.copy()
    df = df[df['cat'] == 'Train']
    local_params = lgb_params.copy()

    # Categorical feature
    for col in categorical_features:
        try:
            df[col] = df[col].astype('category')
        except:
            pass

    all_features = [col for col in list(df) if col not in (remove_features + TARGET)]
    folds = StratifiedKFold(n_splits=cv_split, random_state=SEED, shuffle=True)

    print(all_features)
    
    pred_overall = pd.DataFrame()
    for i, (train_index, val_index) in enumerate(folds.split(df, df['id'])):
        train_data = lgb.Dataset(df.iloc[train_index][all_features], label=df.iloc[train_index][target])
        val_data = lgb.Dataset(df.iloc[val_index][all_features], label=df.iloc[val_index][target])
        temp_df = df.iloc[val_index]

        print('\nCV-{}'.format(i+1))
        seed_everything()
        estimator = lgb.train(local_params,
                              train_data,
                              valid_sets = [train_data, val_data],
                              verbose_eval = verbose)

        temp_df['pred'] = estimator.predict(temp_df[all_features])
        temp_df = temp_df[['row_id','tglpelayanan','kddati2','tkp',target,'pred']]

        print('MAPE CV-{} is {:.2f}%'.format(i+1, mape(temp_df[target], temp_df['pred'])))
        print('MAE CV-{} is {:.2f}'.format(i+1, mae(temp_df[target], temp_df['pred'])))
        
        pred_overall = pred_overall.append(temp_df)
        pred_overall['cv'] = i+1

    print('\nMAPE CV Overall is {:.2f}%'.format(mape(pred_overall[target], pred_overall['pred'])))
    print('MAE CV Overall is {:.2f}'.format(mae(pred_overall[target], pred_overall['pred'])))

    return estimator, pred_overall

In [ ]:
lgb_params = {'boosting_type': 'gbdt', 
              'objective': 'mape',
              # 'objective': 'mape',
              'metric': ['mae'], 
              'learning_rate': 0.1,      
              'subsample': 0.9,       
              'subsample_freq': 1,     
              'num_leaves': 255,            
              'min_data_in_leaf': 255, 
              'feature_fraction': 0.5,
              'n_estimators': 500,   
              'seed': SEED,
              'verbose': -1}
mdl, val_df = process_train(df, cv_split=5)

In [355]:
lgb_params = {'boosting_type': 'gbdt', 
              'objective': 'mae',
              # 'objective': 'mape',
              'metric': ['mae'], 
              'learning_rate': 0.1,      
              'subsample': 0.9,       
              'subsample_freq': 1,     
              'num_leaves': 255,            
              'min_data_in_leaf': 255, 
              'feature_fraction': 0.25,
              'n_estimators': 5000,   
              'seed': SEED,
              'verbose': -1}
mdl, val_df = process_train(df.drop(columns=categorical_features_2), cv_split=5)

['kddati2', 'tkp', 'peserta', 'month', 'year', 'id', 'case2', 'unit_cost2', 'peserta_lag_1', 'case_lag_1', 'unit_cost_lag_1', 'case2_lag_1', 'unit_cost2_lag_1', 'peserta_lag_-1', 'case_lag_-1', 'unit_cost_lag_-1', 'case2_lag_-1', 'unit_cost2_lag_-1', 'ds_case2', 'ds_case_lag_1', 'ds_case2_lag_1', 'ds_case_lag_-1', 'ds_case2_lag_-1']

CV-1
[500]	training's l1: 537.968	valid_1's l1: 662.312
[1000]	training's l1: 486.268	valid_1's l1: 633.133
[1500]	training's l1: 460.92	valid_1's l1: 618.311
[2000]	training's l1: 443.93	valid_1's l1: 608.224
[2500]	training's l1: 430.667	valid_1's l1: 599.91


KeyboardInterrupt: 

In [ ]:
lgb.plot_importance(mdl)

In [ ]:
val_df.sort_values(['kddati2','tkp','tglpelayanan'])

In [ ]:
mape(val_df['case'], val_df['pred'] * 0.975)

In [ ]:
lgb_params = {'boosting_type': 'gbdt', 
              'objective': 'regression',
              # 'objective': 'mape',
              'metric': ['mae'], 
              'learning_rate': 0.05,      
              'subsample': 0.9,       
              'subsample_freq': 1,     
              'num_leaves': 255,            
              'min_data_in_leaf': 255, 
              'feature_fraction': 0.9,
              'n_estimators': 500,   
              'seed': SEED,
              'verbose': -1}
mdl, val_df = process_train(df.drop(columns=categorical_features_2), target=TARGET[1], cv_split=5)

In [ ]:
lgb.plot_importance(mdl)

In [ ]:
lgb_params = {'boosting_type': 'gbdt', 
              'objective': 'regression',
              # 'objective': 'mape',
              'metric': ['mae'], 
              'learning_rate': 0.025,      
              'subsample': 0.9,       
              'subsample_freq': 1,     
              'num_leaves': 255,            
              'min_data_in_leaf': 500, 
              'feature_fraction': 0.95,
              'n_estimators': 1000,   
              'seed': SEED,
              'verbose': -1}
mdl, val_df = process_train(df[['kddati2','tkp','unit_cost_lag_1','unit_cost_lag_-1','cat','id','unit_cost','row_id','tglpelayanan']], target=TARGET[1], cv_split=5)

In [ ]:
lgb_params = {'boosting_type': 'gbdt', 
              'objective': 'mae',
              # 'objective': 'mape',
              'metric': ['mae'], 
              'learning_rate': 0.025,      
              'subsample': 0.9,       
              'subsample_freq': 1,     
              'num_leaves': 255,            
              'min_data_in_leaf': 500, 
              'feature_fraction': 0.95,
              'n_estimators': 1000,   
              'seed': SEED,
              'verbose': -1}
mdl, val_df = process_train(df[['kddati2','tkp','unit_cost_lag_1','unit_cost_lag_-1','cat','id','unit_cost','row_id','tglpelayanan']], target=TARGET[1], cv_split=5)

In [ ]:
df.head()